## Tutorial #5 - Hi, Hydrogen!
The Variational Quantum Eigensolver (VQE) algorithm has been touted as a game-changing near-term quantum algorithm. In particular, VQE is able to efficiently simulate low-energy properties of small molecules. In this challenge, you will calculate the energy of the hydrogen molecule for various molecular charges and bond length combinations.
![chart](./images/5.Hi,Hydrogen_1.png)

**Challenge code**
In the code below, you are given a few functions:

* hydrogen_hamiltonian: This function will return the qubit Hamiltonian of the hydrogen molecule, H, given the coordinates of both hydrogen atoms and the net molecular charge. You'll usually find H with a charge of 0, but here we'll spice it up with a non-zero charge!
* num_electrons: In subsequent functions, we'll need the total number of electrons in the hydrogen molecule we're looking at. With a charge of 0, H usually has just 2 electrons, one per hydrogen atom. Given the charge, how many electrons should H have? You must complete this function.
* hf: The "HF" stands for Hartree–Fock. This function's purpose is calculate the HF approximation — treat every electron as independent, electrons move under a Coulomb potential from the positively charged nuclei, and there's a mean field from the other electrons — for the ground state of the hydrogen molecule we're interested in. We'll use this later, so you must complete this function.
* run_VQE: This function takes the coordinates, charge, generates the HF state, defines a cost function and minimizes it. You must complete this function by:
    * defining the gates within the cost function, using the qml.AllSinglesDoubles template with singles and doubles arguments defined below; and
    * returning what we want to minimize, namely the expectation value of the hydrogen Hamiltonian!


Here are some helpful resources:

* [Building molecular Hamiltonians](https://pennylane.ai/qml/demos/tutorial_quantum_chemistry.html)
* [A brief overview of VQE](https://pennylane.ai/qml/demos/tutorial_vqe.html)
* [Variational Quantum Eigensolver](https://www.youtube.com/watch?v=4Xnxa6tzPeA)
* [Quantum Chemistry documentation](https://docs.pennylane.ai/en/stable/introduction/chemistry.html)

**Input**
As input to this problem, you are given:

* coordinates (list(float)): the x, y, and z coordinates of each hydrogen atom
* charge (int): the charge of the hydrogen molecule. It could be positive, negative, or zero!

**Output**
This code must output the ground state energy (float) of the hydrogen molecule in question.

In [2]:
import json
import pennylane as qml
import pennylane.numpy as np

In [3]:
def hydrogen_hamiltonian(coordinates, charge):
    """Calculates the qubit Hamiltonian of the hydrogen molecule.

    Args:
        coordinates (list(float)): Cartesian coordinates of each hydrogen molecule.
        charge (int): The electric charge given to the hydrogen molecule.

    Returns:
        (qml.Hamiltonian): A PennyLane Hamiltonian.
    """
    return qml.qchem.molecular_hamiltonian(
        ["H", "H"], coordinates, charge, basis="STO-3G"
    )[0]

def num_electrons(charge):
    """The total number of electrons in the hydrogen molecule.

    Args:
        charge (int): The electric charge given to the hydrogen molecule.

    Returns:
        (int): The number of electrons.
    """
    return -charge +2

In [4]:
def hf(electrons, num_qubits):
    """Calculates the Hartree-Fock state of the hydrogen molecule.

    Args:
        electrons (int): The number of electrons in the hydrogen molecule.
        num_qubits (int): The number of qubits needed to represent the hydrogen molecule Hamiltonian.

    Returns:
        (numpy.tensor): The HF state.
    """
    return qml.qchem.hf_state(electrons,num_qubits)

In [11]:
def run_VQE(coordinates, charge):
    """Performs a VQE routine for the given hydrogen molecule.

    Args:
        coordinates (list(float)): Cartesian coordinates of each hydrogen molecule.
        charge (int): The electric charge given to the hydrogen molecule.:

    Returns:
        (float): The expectation value of the hydrogen Hamiltonian.
    """

    hamiltonian = hydrogen_hamiltonian(np.array(coordinates), charge)

    electrons = num_electrons(charge)
    num_qubits = len(hamiltonian.wires)

    hf_state = hf(electrons, num_qubits)
    # singles and doubles are used to make the AllSinglesDoubles template
    singles, doubles = qml.qchem.excitations(electrons, num_qubits)

    dev = qml.device("default.qubit", wires=num_qubits)

    @qml.qnode(dev)
    def cost(weights):
        """A circuit with tunable parameters/weights that measures the expectation value of the hydrogen Hamiltonian.

        Args:
            weights (numpy.array): An array of tunable parameters.

        Returns:
            (float): The expectation value of the hydrogen Hamiltonian.
        """
        qml.AllSinglesDoubles(weights,range(num_qubits),hf_state,singles= singles,doubles=doubles)
        return qml.expval(hamiltonian)

    np.random.seed = 1234
    weights = np.random.normal(
    0, np.pi, len(singles) + len(doubles), requires_grad=True
)
    opt = qml.AdamOptimizer(0.5)

    for n in range(200):
        weights = opt.step(cost, weights)
        if n%20 == 0:
            print(f"step :{n} weights: {weights}")

    return cost(weights)

In [12]:
# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    coordinates, charge = json.loads(test_case_input)
    energy = run_VQE(coordinates, charge)

    return str(energy)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(solution_output, expected_output, rtol=1e-3)

In [13]:
test_cases = [['[[0.0, 0.0, -0.8, 0.0, 0.0, 0.8], -1]', '-0.53168359'], ['[[0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614], 0]', '-1.13618883']]

In [14]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[0.0, 0.0, -0.8, 0.0, 0.0, 0.8], -1]'...
step :0 weights: [-0.07281311]
step :20 weights: [-0.14354935]
step :40 weights: [-0.0152277]
step :60 weights: [-0.00584732]
step :80 weights: [-0.00733932]
step :100 weights: [0.00273649]
step :120 weights: [-0.00053026]
step :140 weights: [-0.00034921]
step :160 weights: [-8.15738702e-05]
step :180 weights: [4.67418045e-05]
Correct!
Running test case 1 with input '[[0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614], 0]'...
step :0 weights: [ 2.19912675 -2.06672029  2.37029751]
step :20 weights: [ 3.35071718 -3.28918813  2.82494773]
step :40 weights: [ 3.21579199 -3.24859084  2.93698698]
step :60 weights: [ 3.1032138  -3.09971825  2.91885848]
step :80 weights: [ 3.14907775 -3.15648364  2.93117547]
step :100 weights: [ 3.1469671  -3.14178759  2.93026826]
step :120 weights: [ 3.14268631 -3.14020613  2.93223064]
step :140 weights: [ 3.14220215 -3.14112868  2.93183717]
step :160 weights: [ 3.14187225 -3.14162176  2.9317363 ]
st